<div align=left style="width: 200; height: 80px; overflow: hidden">
    <img src=http://static1.squarespace.com/static/571446ff60b5e92c3a2b4249/57d8a40b9de4bb459f731cf3/58cb2f229de4bb4a049d38c2/1505340359463/teselaGenlogo.jpg align=right width=200>
</div>

# Hello World! (TEST module)

This notebook is intended to show how to use the Teselagen Python's client to access the TEST module. 

In [1]:
import json
from pathlib import Path
import platform

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from teselagen.api import TeselaGenClient
from teselagen.utils.utils import get_project_root

plt.style.use('seaborn-colorblind')

print(f"python version     : {platform.python_version()}")
print(f"json version       : {json.__version__}")
print(f"pandas version     : {pd.__version__}")
print(f"matplotlib version : {matplotlib.__version__}")

python version     : 3.9.6
json version       : 2.0.9
pandas version     : 1.3.0
matplotlib version : 3.4.2


## Connect and Login

In [2]:
# Connect to your teselagen instance by passing it as the 'host_url' argument of TESTCLient(host_url=host_url)
#host_url = "https://your.teselagen.instance.com"
#client = TESTClient(host_url = host_url)
client = TeselaGenClient()
# The following command will prompt you to type username (email) and password
client.login()
# Alternatively you can add credential information in the arguments
#client.login(username="your@email", password="ZZZ")

Connection Accepted


## Laboratories

Laboratories are used to separate data access into groups of users. When you add an experiment into a lab it will only be seen by users that have access to that Lab. 

Now we list all available laboratories

In [3]:
labs = client.get_laboratories()
display(labs)

[{'id': '70', 'name': 'Example Lab'},
 {'id': '55', 'name': 'Data Science Team'},
 {'id': '72', 'name': 'The Test Lab'}]

At the next cell a lab is selected. The remaining actions of this notebook will be done inside that lab.

In [4]:
# Select a Laboratory
client.select_laboratory(lab_name="Example Lab")

Selected Lab: Example Lab


## Experiments

Experiments represents all the studies inside a laboratory. Each experiment can contain several Assays. The **get_experiments** function returns a list of dictionaries that contain experiments information.

In [5]:
# Get My Experiments from the selected Laboratory.
experiments = client.test.get_experiments()
# Print some of them
for exp in experiments:
    print(exp)

{'id': '37', 'name': 'TD Experiment'}
{'id': '38', 'name': 'TD Glucose Production Experiment'}
{'id': '269', 'name': 'Test Experiment'}
{'id': '322', 'name': 'Python Test Client Experiment'}


In [6]:
# Create a new Experiment in the selected Laboratory.
new_experiment = client.test.create_experiment(experiment_name="Jupyter Experiment")
print(new_experiment)

{'id': '377', 'name': 'Jupyter Experiment'}


## Assays

The assay object allows you to group data obtained from the same *protocol*.

Likewise other getters, the get_assays output can be easily parsed into a pandas dataframe. Here we show you how:

In [7]:
# Get All Assays From all experiments in the selected Laboratory
assays_dataframe = pd.DataFrame(client.test.get_assays())
# Or, alternatively, if you want to use id as index in the dataframe
#assays_dataframe = pd.DataFrame.from_records(client.get_assays(), index='id')
display(assays_dataframe)

,id,name,experiment
0,6,TD Assay 2,"{'id': '37', 'name': 'TD Experiment'}"
1,7,TD Glucose Production Assay,"{'id': '38', 'name': 'TD Glucose Production Ex..."
2,8,TD Glucose Production Assay 2,"{'id': '38', 'name': 'TD Glucose Production Ex..."
3,255,Test Assay,"{'id': '269', 'name': 'Test Experiment'}"
4,256,Test Assay,"{'id': '269', 'name': 'Test Experiment'}"
5,257,Test Assay,"{'id': '269', 'name': 'Test Experiment'}"
6,341,Test Assay,"{'id': '269', 'name': 'Test Experiment'}"
7,342,test Assay,"{'id': '269', 'name': 'Test Experiment'}"
8,361,Python Test Client Assay,"{'id': '322', 'name': 'Python Test Client Expe..."
9,363,Python Test Client Assay,"{'id': '322', 'name': 'Python Test Client Expe..."


You can also create assays using the client

In [8]:
# Create a new Assay within a new experiment
new_assay = client.test.create_assay(experiment_id=new_experiment['id'], assay_name='Jupyter Assay')
print(new_assay)

{'id': '442', 'name': 'Jupyter Assay', 'experiment': {'id': '377', 'name': 'Jupyter Experiment'}}


And you can list all assays from an experiment

In [9]:
# Get All Assays From Experiment with ID="experiment_id"
assays = pd.DataFrame(client.test.get_assays(experiment_id=new_experiment['id']))
display(assays)

,id,name,experiment
0,442,Jupyter Assay,"{'id': '377', 'name': 'Jupyter Experiment'}"


## Files

The TEST module can also store files. Files are shared between users of the same lab and can be associated to specific assays.

In [10]:
# Get All Files info from the selected Laboratory
pd.DataFrame(client.test.get_files_info()).head()
# You can fetch info from a specific assay using the assay_id argument
# display(pd.DataFrame(client.get_files(assay_id=new_assay['id'])))

,id,name,experiment,assay
0,69,Assay A.csv,"{'id': '269', 'name': 'Test Experiment'}","{'id': '341', 'name': 'Test Assay'}"
1,70,Assay A.csv,"{'id': '269', 'name': 'Test Experiment'}","{'id': '342', 'name': 'test Assay'}"
2,76,example_file.csv,None,None
3,77,example_file.csv,None,None
4,78,example_file.csv,None,None


In [11]:
# Upload a File with an Assay.
filepath = str(get_project_root() / "teselagen/api/tests/example_file.csv")
new_file_info = client.test.upload_file(filepath=filepath, assay_id=new_assay['id'])
# Upload a File without an Assay.
#new_file_info = client.upload_file(filepath="./test_module_example.csv")
print(new_file_info)

{'id': '204', 'name': 'test_module_example.csv', 'experiment': {'id': '377', 'name': 'Jupyter Experiment'}, 'assay': {'id': '442', 'name': 'Jupyter Assay'}}


In [12]:
# Gets File data contens
binarydata = client.test.download_file(file_id=new_file_info['id'])
df_file = pd.read_csv(binarydata, sep=",")
df_file.head()

,Line,Teselagen Example Descriptor 1,Teselagen Example Descriptor 2,Teselagen Example Target,Teselagen Example Target Metric
0,1,A0,B1,1,ug/mL
1,2,A0,B2,2,ug/mL
2,3,A0,B3,3,ug/mL
3,4,A0,B5,5,ug/mL
4,5,A0,B6,6,ug/mL


## Delete elements

You can delete an experiment to remove all its assays. Files work differently and you need to explicitly delete them. In the next cells we will download each object specifically:

In [13]:
# Deletes a File
client.test.delete_file(file_id=new_file_info['id'])

{'id': '204'}

In [14]:
# Delete an Assay
client.test.delete_assay(assay_id=new_assay['id'])

{'id': '442'}

In [15]:
# Delete an Experiment from the selected laboratory.
client.test.delete_experiment(experiment_id=new_experiment['id'])

In [16]:
# Delete all experiments with the name Jupyter Experiment
# (if any remains due to uncompleted executions of the notebook)
# experiments = client.get_experiments()
# for exp in experiments:
#     if exp['name'] == "Jupyter Experiment":
#         client.delete_experiment(experiment_id=exp['id'])